In [ ]:
# as you can see current ram is 12GB only, to increase it run below code

In [ ]:
i = []
while True:
  i.append(i)

In [ ]:
# wait for few minutes, after that you will get a prompt that your notebook got crashed, do you want to increase ram ?

In [ ]:
# in the prompt click - yes

In [ ]:
# as you see the ram increase from 12GB to 25GB

In [ ]:
#thanks for watching, keep learning 

In [ ]:
import glob
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Export for Visualization

In [ ]:
# function for restrucuture data
def restructure_data(data):
    import datetime
    newdata = data[data['Fahrplanlage_IST'].notnull()].copy()
    newdata['Ab_ZT_PLAN'] = newdata['Ab_ZT_PLAN'].fillna(newdata['Ab_ZT_PLAN'].shift(1))
    
    #fill time at the end of the each trip table to be the same as departure time
    newdata['Ank_ZT_PLAN'] = 0
    newdata.loc[newdata['Wagen'].shift(-1).notnull(), 'Ank_ZT_PLAN'] = newdata['Ab_ZT_PLAN']
    newdata.loc[~newdata['Wagen'].shift(-1).notnull(), 'Ank_ZT_PLAN'] = newdata['Ab_ZT_PLAN'].shift(-1)
    #newdata['Ank_ZT_PLAN'] = newdata['Ank_ZT_PLAN'].fillna(newdata['Ank_ZT_PLAN'].shift(1))
    
    #find route name start-end station
    newdata['Haltestelle'] = newdata['Haltestelle'].fillna(newdata['Haltestelle'].shift(-1))
    #newdata['S_Station'] = ""
    #newdata['E_Station'] = ""
    newdata.loc[newdata['Wagen'].notnull(), 'S_Station'] = newdata['Haltestelle']
    #newdata.loc[newdata['Fahrplanlage_IST'].shift(1) != 'Fahrplanlage IST', 'S_Station'] = newdata['S_Station'].shift(1)
    
    newdata.loc[newdata['Wagen'].shift(-1).notnull(), 'E_Station'] = newdata['Haltestelle']
    #newdata.loc[newdata['Fahrplanlage_IST'].shift(-1)  != 'Fahrplanlage IST', 'Ank_ZT_PLAN'] = newdata['Ab_ZT_PLAN'].shift(-1)  
    
    
    #rearrage table and create new column for destination bus station
    #newdata = newdata.sort_values(['Wagen', 'Ank_ZT_PLAN'])
    #newdata.loc[newdata['Wagen'].shift(1) != newdata['Wagen'], 'NHaltestelle'] = 'Source'
    newdata.loc[newdata['Wagen'].shift(-1).isnull(), 'NHaltestelle'] = newdata['Haltestelle'].shift(-1)
    #newdata['NHaltestelle'] = newdata['Haltestelle'].shift(-1)
    newdata['NHaltestelle'] = newdata['NHaltestelle'].fillna(newdata['Haltestelle'].shift(-1))
    newdata['Node A to Node B']= newdata['Haltestelle']+"|"+ newdata['NHaltestelle']
    
    
    #Fill empty with above value
    #newdata['S_Station'].fillna(method='ffill', inplace=True)
    #newdata.fillna({'Datum':'ffill', 'UML':'ffill'}, inplace=True)
    #newdata.update(newdata[['Datum', 'UML']].fillna(method='ffill', inplace=True))
    newdata['Datum'].fillna(method='ffill', inplace=True)
    newdata['UML'].fillna(method='ffill', inplace=True)
    #newdata = newdata[newdata['Fahrplanlage_IST']!='Fahrplanlage IST']
    

    #prevent wrong date data
    newdata['date'] = newdata['Datum'].astype(str).str[:10].replace('2020.', '2019.', regex=True)
    #newdata = newdata.drop('Datum', axis = 1)
    
           
    #change characteristic of departure and arrival
    newdata['Ab_ZT_PLAN'] = pd.to_datetime(newdata['date'] + ' ' + newdata['Ab_ZT_PLAN'], format='%d.%m.%Y %H:%M', errors='ignore')
    newdata['Ank_ZT_PLAN'] = pd.to_datetime(newdata['date'] + ' ' + newdata['Ank_ZT_PLAN'], format='%d.%m.%Y %H:%M', errors='ignore')
    
    newdata['Linie'] = newdata['Linie'].astype(str).str[:-2]
    newdata['UML'] = newdata['UML'].astype(str).str[:-2]
    #set journey ref
    newdata.loc[newdata['Wagen'].notnull(), 'JourneyRef'] = newdata['Ab_ZT_PLAN'].astype(str) + newdata['UML'] + newdata['Linie']
    newdata.loc[newdata['Wagen'].notnull(), 'S_Time'] = newdata['Ab_ZT_PLAN']
    newdata.loc[newdata['Wagen'].shift(-1).notnull(), 'E_Time'] = newdata['Ank_ZT_PLAN']

    #Fill empty with below value
    newdata['E_Station'].fillna(method='bfill', inplace=True)
    newdata['E_Time'].fillna(method='bfill', inplace=True)
    #Fill empty with above value
    newdata.fillna(method='ffill', inplace=True)
    
    newdata['Wagen'] = newdata['Wagen'].astype(str).str[:-2]

    newdata['Fahrplanlage_IST'] = pd.to_numeric( newdata['Fahrplanlage_IST'].astype(str).replace(',', '', regex = True))
    #actual time calculate from minute late
    newdata['Ab_ZT_ACTUAL'] = newdata['Ab_ZT_PLAN']+pd.to_timedelta(newdata['Fahrplanlage_IST'], unit='min')
    
    newdata.loc[newdata['Ab_ZT_PLAN']  == newdata['Ank_ZT_PLAN'], 'Ank_ZT_ACTUAL'] = newdata['Ab_ZT_ACTUAL']
    newdata['Ank_ZT_ACTUAL'] = newdata['Ank_ZT_ACTUAL'].fillna(newdata['Ab_ZT_ACTUAL'].shift(-1))
    #newdata['Ank_ZT_ACTUAL'] = newdata['Ab_ZT_ACTUAL'].shift(-1)
    
    #calculate driving time and distance
    newdata['Driving_time_s'] = (pd.to_datetime(newdata['Ank_ZT_ACTUAL']) - pd.to_datetime(newdata['Ab_ZT_ACTUAL'])).dt.total_seconds() 
    newdata['Distance_m'] = pd.to_numeric( newdata['v_IST'].astype(str).replace(',|-', '', regex = True)) * 10/36 * newdata['Driving_time_s']
    newdata['Driving_time_s_p'] = (pd.to_datetime(newdata['Ank_ZT_PLAN']) - pd.to_datetime(newdata['Ab_ZT_PLAN'])).dt.total_seconds() 
    newdata['Distance_m_p'] = pd.to_numeric( newdata['v_PLAN'].astype(str).replace(',|-', '', regex = True)) * 10/36 * newdata['Driving_time_s_p']
    
    selectdata = newdata[['JourneyRef', 'UML', 'Wagen', 'Linie','S_Time', 'S_Station', 'E_Station', 'E_Time','Driving_time_s', 'Distance_m', 'Driving_time_s_p', 'Distance_m_p']]
    gb = selectdata.groupby(['JourneyRef','UML', 'Wagen', 'Linie','S_Time', 'S_Station', 'E_Station', 'E_Time']).sum().reset_index()
    return gb #newdata

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fahrten BBD BBW 01.03.2019-30.09.2019/bkf06_t.csv', sep=',',skiprows=3,
                  names = ['Datum','UML','Wagen','Ab_ZT_PLAN','Linie','Haltestelle','Fahrplanlage_IST','v_PLAN','v_IST_S2S1','v_netto_S2S1','FFVP_GESCHWINDIGKEIT','v_IST'
                          ])
#df['date'] = date_list[i]
#df['line'] = line_list[i]

df = restructure_data(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df

,JourneyRef,UML,Wagen,Linie,S_Time,S_Station,E_Station,E_Time,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
0,2019-09-01 00:00:007705825,77058,7337,25,2019-09-01 00:00:00,Max-Brauer-Allee (Mitte),S Holstenstraße,2019-09-01 23:58:00,86310.0,14357.500000,86280.0,13845.000000
1,2019-09-01 00:00:00771197,77119,4706,7,2019-09-01 00:00:00,Gründgensstraße (West),Eichenlohweg,2019-09-01 23:59:00,86328.0,4507.000000,86340.0,4620.000000
2,2019-09-01 00:00:0077122173,77122,1731,173,2019-09-01 00:00:00,Schumannstraße,Beethovenstraße,2019-09-01 23:58:00,86352.0,4018.666667,86280.0,3985.000000
3,2019-09-01 00:00:007712317,77123,3925,17,2019-09-01 00:00:00,U St.Pauli,Michaeliskirche,2019-09-01 23:58:00,86256.0,6969.333333,86280.0,6711.666667
4,2019-09-01 00:00:007712417,77124,6911,17,2019-09-01 00:00:00,Mundsburger Brücke,AK St.Georg,2019-09-01 23:57:00,86250.0,597341.666667,86220.0,645945.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
68198,2019-09-24 22:11:0077014174,77014,6011,174,2019-09-24 22:11:00,U Volksdorf,S Poppenbüttel (Wentzelplatz),2019-09-24 22:25:00,918.0,7416.833333,840.0,7643.333333
68199,2019-09-24 22:26:0077014174,77014,6011,174,2019-09-24 22:26:00,S Poppenbüttel (Wentzelplatz),S Poppenbüttel (Wentzelplatz),2019-09-24 22:26:00,0.0,0.000000,0.0,0.000000
68200,2019-09-24 22:37:00770016,77001,7283,6,2019-09-24 22:37:00,Am Sandtorkai,Gertigstraße,2019-09-24 22:58:00,1194.0,5289.166667,1200.0,5260.000000
68201,2019-09-24 22:37:00770016,77001,7283,n,2019-09-24 22:37:00,Am Sandtorkai,Gertigstraße,2019-09-24 22:58:00,90.0,392.500000,60.0,0.000000


In [ ]:
path =r'/content/drive/My Drive/Colab Notebooks/Fahrten BBD BBW 01.03.2019-30.09.2019' # use your path
allFiles = glob.glob(path + "/*.csv")

date_list = [allFile[-15:-5] for allFile in allFiles]
#line_list = [allFile[-21:-16] for allFile in allFiles]

list_ = []
i=0


for file_ in allFiles:
    df = pd.read_csv(file_, sep=',',skiprows=3,
                      names = ['Datum','UML','Wagen','Ab_ZT_PLAN','Linie','Haltestelle','Fahrplanlage_IST','v_PLAN','v_IST_S2S1','v_netto_S2S1','FFVP_GESCHWINDIGKEIT','v_IST'
                              ])
    #df['date'] = date_list[i]
    #df['line'] = line_list[i]

    df = restructure_data(df)
    list_.append(df)
    i+=1

data = pd.concat(list_, axis = 0, ignore_index = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data[-50:]

,JourneyRef,UML,Wagen,Linie,S_Time,S_Station,E_Station,E_Time,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
1403457,2019-10-24 22:55:007707623,77076,1912,23,2019-10-24 22:55:00,U Billstedt,U Niendorf Markt,2019-10-24 23:57:00,3096.0,18866.666667,3480.0,18848.333333
1403458,2019-10-24 22:55:007707623,77076,1912,n,2019-10-24 22:55:00,U Billstedt,U Niendorf Markt,2019-10-24 23:57:00,414.0,1368.500000,240.0,1453.333333
1403459,2019-10-24 22:56:00772777,77277,7526,7,2019-10-24 22:56:00,Borchertring,Borchertring,2019-10-24 22:56:00,0.0,0.000000,0.0,0.000000
1403460,2019-10-24 22:57:00772655,77265,4927,5,2019-10-24 22:57:00,A Burgwedel,Hauptbahnhof/Mönckebergstraße,2019-10-24 23:37:00,2532.0,13690.333333,2400.0,13555.000000
1403461,2019-10-24 22:58:0077089177,77089,1109,177,2019-10-24 22:58:00,Borchertring,Borchertring,2019-10-24 22:58:00,0.0,0.000000,0.0,0.000000
1403462,2019-10-24 22:58:00772377,77237,4805,7,2019-10-24 22:58:00,Borchertring,U S Barmbek,2019-10-24 23:11:00,744.0,4516.333333,720.0,4633.333333
1403463,2019-10-24 22:58:00772377,77237,4805,n,2019-10-24 22:58:00,Borchertring,U S Barmbek,2019-10-24 23:11:00,36.0,314.000000,60.0,288.333333
1403464,2019-10-24 22:58:00772616,77261,7916,6,2019-10-24 22:58:00,Am Sandtorkai,Gertigstraße,2019-10-24 23:19:00,1098.0,4025.666667,1200.0,6143.333333
1403465,2019-10-24 22:58:00772616,77261,7916,n,2019-10-24 22:58:00,Am Sandtorkai,Gertigstraße,2019-10-24 23:19:00,156.0,186.333333,60.0,0.000000
1403466,2019-10-24 22:58:0077272368,77272,1993,368,2019-10-24 22:58:00,U Berne (Berner Heerweg),Bf. Rahlstedt (Doberaner Weg),2019-10-24 23:20:00,1356.0,7196.333333,1320.0,7238.333333


In [ ]:
gb = data.groupby(['JourneyRef','UML', 'Wagen', 'Linie','S_Time', 'S_Station', 'E_Station', 'E_Time']).sum().reset_index()
gb

,JourneyRef,UML,Wagen,Linie,S_Time,S_Station,E_Station,E_Time,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
0,2019-05-01 00:00:007705325,77053,7435,25,2019-05-01 00:00:00,Max-Brauer-Allee (Mitte),S Holstenstraße,2019-05-01 23:58:00,86340.0,13377.166667,86280.0,378870.000000
1,2019-05-01 00:00:0077108172,77108,1314,172,2019-05-01 00:00:00,Herderstraße,Jarrestraße (Kampnagel),2019-05-01 23:58:00,86256.0,8866.000000,86280.0,9046.666667
2,2019-05-01 00:00:00771097,77109,7308,7,2019-05-01 00:00:00,Gründgensstraße (West),Eichenlohweg,2019-05-01 23:59:00,86316.0,4407.666667,86340.0,4620.000000
3,2019-05-01 00:00:0077113173,77113,1117,173,2019-05-01 00:00:00,Beethovenstraße,U Mundsburg,2019-05-01 23:58:00,86298.0,3987.500000,86280.0,3618.333333
4,2019-05-01 00:00:007711617,77116,7618,17,2019-05-01 00:00:00,U St.Pauli,Michaeliskirche,2019-05-01 23:58:00,86232.0,8369.833333,86280.0,8176.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
67423,2019-05-23 22:02:0077017172,77017,2735,172,2019-05-23 22:02:00,Lentersweg,U S Barmbek,2019-05-23 22:26:00,1422.0,7817.000000,1440.0,7975.000000
67424,2019-05-23 22:14:0077047171,77047,1725,171,2019-05-23 22:14:00,U Farmsen,U S Barmbek,2019-05-23 22:40:00,1608.0,8894.333333,1560.0,9116.666667
67425,2019-05-23 22:41:0077047171,77047,1725,171,2019-05-23 22:41:00,U S Barmbek,U S Barmbek,2019-05-23 22:41:00,0.0,0.000000,0.0,0.000000
67426,2019-05-23 22:46:0077017172,77017,2735,172,2019-05-23 22:46:00,U S Barmbek,Im Grünen Grunde,2019-05-23 22:58:00,720.0,4368.333333,720.0,4441.666667


In [ ]:
data.loc[data['JourneyRef']=='2019-05-23 22:59:0077017172']

,JourneyRef,UML,Wagen,Linie,S_Time,S_Station,E_Station,E_Time,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
1040092,2019-05-23 22:59:0077017172,77017,2735,172,2019-05-23 22:59:00,Im Grünen Grunde,Im Grünen Grunde,2019-05-23 22:59:00,0.0,0.0,0.0,0.0


In [ ]:
# function for aggregate data
def aggregate_data(data):
    import datetime
    newdata = data[data['Fahrplanlage_IST'].notnull()].copy()
    newdata['Ab_ZT_PLAN'] = newdata['Ab_ZT_PLAN'].fillna(newdata['Ab_ZT_PLAN'].shift(1))
    '''
    #fill time at the end of the each trip table to be the same as departure time
    newdata['Ank_ZT_PLAN'] = 0
    newdata.loc[newdata['Fahrplanlage_IST'].shift(-1)  == 'Fahrplanlage IST', 'Ank_ZT_PLAN'] = newdata['Ab_ZT_PLAN']
    newdata.loc[newdata['Fahrplanlage_IST'].shift(-1)  != 'Fahrplanlage IST', 'Ank_ZT_PLAN'] = newdata['Ab_ZT_PLAN'].shift(-1)
    newdata['Ank_ZT_PLAN'] = newdata['Ank_ZT_PLAN'].fillna(newdata['Ank_ZT_PLAN'].shift(1))
    
    #find route name start-end station
    newdata['Haltestelle'] = newdata['Haltestelle'].fillna(newdata['Haltestelle'].shift(-1))
    #newdata['S_Station'] = ""
    #newdata['E_Station'] = ""
    newdata.loc[newdata['Wagen'].notnull(), 'S_Station'] = newdata['Haltestelle']
    #newdata.loc[newdata['Fahrplanlage_IST'].shift(1) != 'Fahrplanlage IST', 'S_Station'] = newdata['S_Station'].shift(1)
    
    newdata.loc[newdata['Wagen'].shift(-1).notnull(), 'E_Station'] = newdata['Haltestelle']
    #newdata.loc[newdata['Fahrplanlage_IST'].shift(-1)  != 'Fahrplanlage IST', 'Ank_ZT_PLAN'] = newdata['Ab_ZT_PLAN'].shift(-1)  

    #rearrage table and create new column for destination bus station
    #newdata = newdata.sort_values(['Wagen', 'Ank_ZT_PLAN'])
    #newdata.loc[newdata['Wagen'].shift(1) != newdata['Wagen'], 'NHaltestelle'] = 'Source'
    newdata.loc[newdata['Wagen'].shift(-1).isnull(), 'NHaltestelle'] = newdata['Haltestelle'].shift(-1)
    #newdata['NHaltestelle'] = newdata['Haltestelle'].shift(-1)
    newdata['NHaltestelle'] = newdata['NHaltestelle'].fillna('Source')
    newdata['Node A to Node B']= newdata['Haltestelle']+"|"+ newdata['NHaltestelle']
    
    #Fill empty with below value
    newdata['E_Station'].fillna(method='bfill', inplace=True)
    #Fill empty with above value
    newdata.fillna(method='ffill', inplace=True)
    #newdata = newdata[newdata['Fahrplanlage_IST']!='Fahrplanlage IST']
    newdata['Linie'] = newdata['Linie'].astype(object)
    newdata['UML'] = newdata['UML'].astype(object)
    newdata['Wagen'] = newdata['Wagen'].astype(object)

    #prevent wrong date data
    newdata['date'] = newdata['Datum'].astype(str).str[:10].replace('2020.', '2019.', regex=True)
    #newdata = newdata.drop('Datum', axis = 1)
    
           
    #change characteristic of departure and arrival
    newdata['Ab_ZT_PLAN'] = pd.to_datetime(newdata['date'] + ' ' + newdata['Ab_ZT_PLAN'], format='%d.%m.%Y %H:%M', errors='ignore')
    newdata['Ank_ZT_PLAN'] = pd.to_datetime(newdata['date'] + ' ' + newdata['Ank_ZT_PLAN'], format='%d.%m.%Y %H:%M', errors='ignore')
    
    newdata['Fahrplanlage_IST'] = pd.to_numeric( newdata['Fahrplanlage_IST'].astype(str).replace(',', '', regex = True))
    #actual time calculate from minute late
    newdata['Ab_ZT_ACTUAL'] = newdata['Ab_ZT_PLAN']+pd.to_timedelta(newdata['Fahrplanlage_IST'], unit='min')
    
    newdata.loc[newdata['Ab_ZT_PLAN']  == newdata['Ank_ZT_PLAN'], 'Ank_ZT_ACTUAL'] = newdata['Ab_ZT_ACTUAL']
    newdata['Ank_ZT_ACTUAL'] = newdata['Ank_ZT_ACTUAL'].fillna(newdata['Ab_ZT_ACTUAL'].shift(-1))
    #newdata['Ank_ZT_ACTUAL'] = newdata['Ab_ZT_ACTUAL'].shift(-1)
    
    #calculate driving time and distance
    newdata['Driving_time_s'] = (pd.to_datetime(newdata['Ank_ZT_ACTUAL']) - pd.to_datetime(newdata['Ab_ZT_ACTUAL'])).dt.total_seconds() 
    newdata['Distance_m'] = pd.to_numeric( newdata['v_IST'].astype(str).replace(',|-', '', regex = True)) * 10/36 * newdata['Driving_time_s']
    newdata['Driving_time_s_p'] = (pd.to_datetime(newdata['Ank_ZT_PLAN']) - pd.to_datetime(newdata['Ab_ZT_PLAN'])).dt.total_seconds() 
    newdata['Distance_m_p'] = pd.to_numeric( newdata['v_PLAN'].astype(str).replace(',|-', '', regex = True)) * 10/36 * newdata['Driving_time_s_p']
    '''
    return newdata

In [ ]:
path =r'/content/drive/My Drive/Colab Notebooks/Fahrten BBD BBW 01.03.2019-30.09.2019' # use your path
allFiles = glob.glob(path + "/*.csv")

date_list = [allFile[-15:-5] for allFile in allFiles]
#line_list = [allFile[-21:-16] for allFile in allFiles]

list_ = []
i=0


for file_ in allFiles:
    df = pd.read_csv(file_, sep=',',skiprows=3,
                      names = ['Datum','UML','Wagen','Ab_ZT_PLAN','Linie','Haltestelle','Fahrplanlage_IST','v_PLAN','v_IST_S2S1','v_netto_S2S1','FFVP_GESCHWINDIGKEIT','v_IST'
                              ])
    #df['date'] = date_list[i]
    #df['line'] = line_list[i]

    df = aggregate_data(df)
    list_.append(df)
    i+=1

agg_data = pd.concat(list_, axis = 0, ignore_index = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
agg_data[:50]

,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST
0,01.05.2019,77001.0,7309.0,03:58,25.0,U Burgstraße,0.8,32.8,19.9,17.1,15,15.7
1,NaN,NaN,NaN,03:59,25.0,Carl-Petersen-Straße,2,10.9,655.2,3.6,0,46.8
2,NaN,NaN,NaN,04:00,25.0,S Landwehr (Ramazan-Avci-Platz),1.1,24.4,-,-,30,14.4
3,NaN,NaN,NaN,04:01,25.0,U Wartenau,1.8,27.8,45.1,39.7,0,39.8
4,NaN,NaN,NaN,04:02,25.0,Uferstraße,1.5,16.3,46.5,36.5,38,20.3
5,NaN,NaN,NaN,04:03,25.0,U Mundsburg,1.8,20.0,37.6,33.2,0,20.9
6,NaN,NaN,NaN,04:05,25.0,Beethovenstraße,1.4,22.0,17.9,15.0,0,14.5
7,NaN,NaN,NaN,04:10,25.0,Gertigstraße,1.6,27.1,42.7,32.9,0,31.2
8,NaN,NaN,NaN,04:11,25.0,Goldbekplatz,1.6,25.0,39.5,34.2,2,25.9
9,NaN,NaN,NaN,04:12,25.0,Dorotheenstraße,1.4,11.6,31.7,27.5,26,23.2


# Export full data

In [ ]:
#data.to_csv('restructured_hochbahn.csv')
data.to_csv('IST_19.csv')
!cp IST_19.csv "/content/drive/My Drive/Colab Notebooks/Thesis"

## Filter null value

In [ ]:
#filter v_IST == null (11416) or v_IST_S2S1 == null (14147)
data_ist_fil = data[pd.notna(data.v_IST) & pd.notna(data.v_IST_S2S1)]
#filter Distance_m == null or <0 (11621)
data_ist_fil = data_ist_fil[pd.notna(data_ist_fil.Distance_m) & data_ist_fil.Distance_m >= 0]
#filter Distance_m_p == null or == 0 (12799)
data_ist_fil = data_ist_fil[pd.notna(data_ist_fil.Distance_m_p) & data_ist_fil.Distance_m_p != 0]
data_ist_fil

,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST,Ank_ZT_PLAN,S_Station,E_Station,NHaltestelle,Node A to Node B,date,Ab_ZT_ACTUAL,Ank_ZT_ACTUAL,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
0,01.05.2019,88001,1506,2019-05-01 04:06:00,171,U Farmsen,-0.1,19.0,12.9,10.4,23,12.1,2019-05-01 04:07:00,U Farmsen,Straßburger Platz,Eissporthalle,U Farmsen|Eissporthalle,01.05.2019,2019-05-01 04:05:54,2019-05-01 04:07:24,90.0,302.500000,60.0,316.666667
1,01.05.2019,88001,1506,2019-05-01 04:07:00,171,Eissporthalle,0.4,25.3,47.4,42.5,35,41.0,2019-05-01 04:08:00,U Farmsen,Straßburger Platz,Eckerkoppel,Eissporthalle|Eckerkoppel,01.05.2019,2019-05-01 04:07:24,2019-05-01 04:08:06,42.0,478.333333,60.0,421.666667
2,01.05.2019,88001,1506,2019-05-01 04:08:00,171,Eckerkoppel,0.1,36.1,49.2,45.2,35,44.2,2019-05-01 04:09:00,U Farmsen,Straßburger Platz,Ebeersreye,Eckerkoppel|Ebeersreye,01.05.2019,2019-05-01 04:08:06,2019-05-01 04:08:54,48.0,589.333333,60.0,601.666667
3,01.05.2019,88001,1506,2019-05-01 04:09:00,171,Ebeersreye,-0.1,20.1,14.3,13.2,38,13.9,2019-05-01 04:11:00,U Farmsen,Straßburger Platz,Rentenversicherung Nord,Ebeersreye|Rentenversicherung Nord,01.05.2019,2019-05-01 04:08:54,2019-05-01 04:11:48,174.0,671.833333,120.0,670.000000
4,01.05.2019,88001,1506,2019-05-01 04:11:00,171,Rentenversicherung Nord,0.8,19.1,47.9,41.4,45,30.2,2019-05-01 04:12:00,U Farmsen,Straßburger Platz,Sylter Weg,Rentenversicherung Nord|Sylter Weg,01.05.2019,2019-05-01 04:11:48,2019-05-01 04:12:30,42.0,352.333333,60.0,318.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22048748,24.10.2019,77279,1934,2019-10-24 16:24:00,172,U Mundsburg,0.2,22.5,29.3,25.1,0,18.2,2019-10-24 16:25:00,Lentersweg,Mundsburger Brücke,Uferstraße,U Mundsburg|Uferstraße,24.10.2019,2019-10-24 16:24:12,2019-10-24 16:25:24,72.0,364.000000,60.0,375.000000
22048749,24.10.2019,77279,1934,2019-10-24 16:25:00,172,Uferstraße,0.4,28.5,38.0,34.1,0,30.0,2019-10-24 16:26:00,Lentersweg,Mundsburger Brücke,U Uhlandstraße,Uferstraße|U Uhlandstraße,24.10.2019,2019-10-24 16:25:24,2019-10-24 16:26:12,48.0,400.000000,60.0,475.000000
22048750,24.10.2019,77279,1934,2019-10-24 16:26:00,172,U Uhlandstraße,0.2,7.7,18.8,15.0,27,16.2,2019-10-24 16:29:00,Lentersweg,Mundsburger Brücke,Mundsburger Brücke,U Uhlandstraße|Mundsburger Brücke,24.10.2019,2019-10-24 16:26:12,2019-10-24 16:27:24,72.0,324.000000,180.0,385.000000
22048752,24.10.2019,77279,1934,2019-10-24 16:39:00,173,Mundsburger Brücke,0.3,12.4,15.3,13.8,0,8.8,2019-10-24 16:42:00,Mundsburger Brücke,Mundsburger Brücke,U Mundsburg,Mundsburger Brücke|U Mundsburg,24.10.2019,2019-10-24 16:39:18,2019-10-24 16:43:48,270.0,660.000000,180.0,620.000000


## Export data

In [ ]:
#data.to_csv('restructured_hochbahn.csv')
data_ist_fil.to_csv('IST_filter_19.csv')
!cp IST_filter_19.csv "/content/drive/My Drive/Colab Notebooks/Thesis"

# Import to filter line 28

In [ ]:
data_ist_fil = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Thesis/IST_filter_19.csv', sep=',',skiprows=3,
                      names = ['Datum','UML','Wagen','Ab_ZT_PLAN','Linie',
                               'Haltestelle','Fahrplanlage_IST','v_PLAN',
                               'v_IST_S2S1','v_netto_S2S1','FFVP_GESCHWINDIGKEIT',
                               'v_IST', 'Ank_ZT_PLAN', 'S_Station',	'E_Station',
                               'NHaltestelle',	'Node A to Node B',	'date',	
                               'Ab_ZT_ACTUAL',	'Ank_ZT_ACTUAL',	'Driving_time_s',	
                               'Distance_m',	'Driving_time_s_p',	'Distance_m_p'
                              ])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_ist_fil

,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST,Ank_ZT_PLAN,S_Station,E_Station,NHaltestelle,Node A to Node B,date,Ab_ZT_ACTUAL,Ank_ZT_ACTUAL,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
2,01.05.2019,88001.0,1506.0,2019-05-01 04:08:00,171,Eckerkoppel,0.1,36.1,49.2,45.2,35,44.2,2019-05-01 04:09:00,U Farmsen,Straßburger Platz,Ebeersreye,Eckerkoppel|Ebeersreye,01.05.2019,2019-05-01 04:08:06,2019-05-01 04:08:54,48.0,589.333333,60.0,601.666667
3,01.05.2019,88001.0,1506.0,2019-05-01 04:09:00,171,Ebeersreye,-0.1,20.1,14.3,13.2,38,13.9,2019-05-01 04:11:00,U Farmsen,Straßburger Platz,Rentenversicherung Nord,Ebeersreye|Rentenversicherung Nord,01.05.2019,2019-05-01 04:08:54,2019-05-01 04:11:48,174.0,671.833333,120.0,670.000000
4,01.05.2019,88001.0,1506.0,2019-05-01 04:11:00,171,Rentenversicherung Nord,0.8,19.1,47.9,41.4,45,30.2,2019-05-01 04:12:00,U Farmsen,Straßburger Platz,Sylter Weg,Rentenversicherung Nord|Sylter Weg,01.05.2019,2019-05-01 04:11:48,2019-05-01 04:12:30,42.0,352.333333,60.0,318.333333
5,01.05.2019,88001.0,1506.0,2019-05-01 04:12:00,171,Sylter Weg,0.5,28.4,18.5,16.2,0,16.3,2019-05-01 04:13:00,U Farmsen,Straßburger Platz,Am Stadtrand,Sylter Weg|Am Stadtrand,01.05.2019,2019-05-01 04:12:30,2019-05-01 04:14:06,96.0,434.666667,60.0,473.333333
6,01.05.2019,88001.0,1506.0,2019-05-01 04:13:00,171,Am Stadtrand,1.1,21.2,50.8,44.5,43,43.8,2019-05-01 04:14:00,U Farmsen,Straßburger Platz,Helbingtwiete,Am Stadtrand|Helbingtwiete,01.05.2019,2019-05-01 04:14:06,2019-05-01 04:14:36,30.0,365.000000,60.0,353.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22048748,24.10.2019,77279.0,1934.0,2019-10-24 16:24:00,172,U Mundsburg,0.2,22.5,29.3,25.1,0,18.2,2019-10-24 16:25:00,Lentersweg,Mundsburger Brücke,Uferstraße,U Mundsburg|Uferstraße,24.10.2019,2019-10-24 16:24:12,2019-10-24 16:25:24,72.0,364.000000,60.0,375.000000
22048749,24.10.2019,77279.0,1934.0,2019-10-24 16:25:00,172,Uferstraße,0.4,28.5,38.0,34.1,0,30.0,2019-10-24 16:26:00,Lentersweg,Mundsburger Brücke,U Uhlandstraße,Uferstraße|U Uhlandstraße,24.10.2019,2019-10-24 16:25:24,2019-10-24 16:26:12,48.0,400.000000,60.0,475.000000
22048750,24.10.2019,77279.0,1934.0,2019-10-24 16:26:00,172,U Uhlandstraße,0.2,7.7,18.8,15.0,27,16.2,2019-10-24 16:29:00,Lentersweg,Mundsburger Brücke,Mundsburger Brücke,U Uhlandstraße|Mundsburger Brücke,24.10.2019,2019-10-24 16:26:12,2019-10-24 16:27:24,72.0,324.000000,180.0,385.000000
22048752,24.10.2019,77279.0,1934.0,2019-10-24 16:39:00,173,Mundsburger Brücke,0.3,12.4,15.3,13.8,0,8.8,2019-10-24 16:42:00,Mundsburger Brücke,Mundsburger Brücke,U Mundsburg,Mundsburger Brücke|U Mundsburg,24.10.2019,2019-10-24 16:39:18,2019-10-24 16:43:48,270.0,660.000000,180.0,620.000000


In [ ]:
#filter ONLY LINE 28
data_ist_fil = data_ist_fil[data_ist_fil.Linie == 28].reset_index()
data_ist_fil

,index,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST,Ank_ZT_PLAN,S_Station,E_Station,NHaltestelle,Node A to Node B,date,Ab_ZT_ACTUAL,Ank_ZT_ACTUAL,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
0,1588,01.05.2019,88008.0,1772.0,2019-05-01 09:41:00,28,U Wandsbek Markt,0.5,18.9,12.8,11.9,15,12.2,2019-05-01 09:44:00,U Wandsbek Markt,S Rübenkamp,Wandsbeker Allee,U Wandsbek Markt|Wandsbeker Allee,01.05.2019,2019-05-01 09:41:30,2019-05-01 09:46:18,288.0,976.000000,180.0,945.000000
1,1589,01.05.2019,88008.0,1772.0,2019-05-01 09:44:00,28,Wandsbeker Allee,2.3,10.7,17.1,17.1,0,12.6,2019-05-01 09:46:00,U Wandsbek Markt,S Rübenkamp,U Straßburger Straße,Wandsbeker Allee|U Straßburger Straße,01.05.2019,2019-05-01 09:46:18,2019-05-01 09:48:00,102.0,357.000000,120.0,356.666667
2,1590,01.05.2019,88008.0,1772.0,2019-05-01 09:46:00,28,U Straßburger Straße,2.0,31.4,33.7,25.6,0,14.1,2019-05-01 09:47:00,U Wandsbek Markt,S Rübenkamp,U Alter Teichweg,U Straßburger Straße|U Alter Teichweg,01.05.2019,2019-05-01 09:48:00,2019-05-01 09:50:18,138.0,540.500000,60.0,523.333333
3,1591,01.05.2019,88008.0,1772.0,2019-05-01 09:47:00,28,U Alter Teichweg,3.3,28.6,32.7,29.7,0,27.0,2019-05-01 09:49:00,U Wandsbek Markt,S Rübenkamp,Habichtstraße (Mitte),U Alter Teichweg|Habichtstraße (Mitte),01.05.2019,2019-05-01 09:50:18,2019-05-01 09:52:12,114.0,855.000000,120.0,953.333333
4,1592,01.05.2019,88008.0,1772.0,2019-05-01 09:49:00,28,Habichtstraße (Mitte),3.2,11.3,24.3,9.3,40,14.2,2019-05-01 09:50:00,U Wandsbek Markt,S Rübenkamp,U Habichtstraße,Habichtstraße (Mitte)|U Habichtstraße,01.05.2019,2019-05-01 09:52:12,2019-05-01 09:53:12,60.0,236.666667,60.0,188.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673736,22038712,24.10.2019,77200.0,1612.0,2019-10-24 18:53:00,28,U Habichtstraße,14.7,19.8,4.7,3.8,0,4.1,2019-10-24 18:54:00,S Rübenkamp,Wandsbeker Allee,Habichtstraße (Mitte),U Habichtstraße|Habichtstraße (Mitte),24.10.2019,2019-10-24 19:07:42,2019-10-24 19:12:18,276.0,314.333333,60.0,330.000000
673737,22038713,24.10.2019,77200.0,1612.0,2019-10-24 18:54:00,28,Habichtstraße (Mitte),18.3,16.1,7.4,6.9,0,6.7,2019-10-24 18:57:00,S Rübenkamp,Wandsbeker Allee,U Alter Teichweg,Habichtstraße (Mitte)|U Alter Teichweg,24.10.2019,2019-10-24 19:12:18,2019-10-24 19:19:42,444.0,826.333333,180.0,805.000000
673738,22038714,24.10.2019,77200.0,1612.0,2019-10-24 18:57:00,28,U Alter Teichweg,22.7,20.9,8.6,6.8,0,6.9,2019-10-24 18:59:00,S Rübenkamp,Wandsbeker Allee,U Straßburger Straße,U Alter Teichweg|U Straßburger Straße,24.10.2019,2019-10-24 19:19:42,2019-10-24 19:25:30,348.0,667.000000,120.0,696.666667
673739,22038715,24.10.2019,77200.0,1612.0,2019-10-24 18:59:00,28,U Straßburger Straße,26.5,18.1,-,-,19,11.9,2019-10-24 19:00:00,S Rübenkamp,Wandsbeker Allee,Wandsbeker Allee,U Straßburger Straße|Wandsbeker Allee,24.10.2019,2019-10-24 19:25:30,2019-10-24 19:27:00,90.0,297.500000,60.0,301.666667


In [ ]:
#data.to_csv('restructured_hochbahn.csv')
data_ist_fil.to_csv('IST_filter_Line28.csv')
!cp IST_filter_Line28.csv "/content/drive/My Drive/Colab Notebooks/Thesis"

In [ ]:
data_ist_fil = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Thesis/IST_filter_Line28.csv', sep=',',skiprows=3,
                      names = ['Datum','UML','Wagen','Ab_ZT_PLAN','Linie',
                               'Haltestelle','Fahrplanlage_IST','v_PLAN',
                               'v_IST_S2S1','v_netto_S2S1','FFVP_GESCHWINDIGKEIT',
                               'v_IST', 'Ank_ZT_PLAN', 'S_Station',	'E_Station',
                               'NHaltestelle',	'Node A to Node B',	'date',	
                               'Ab_ZT_ACTUAL',	'Ank_ZT_ACTUAL',	'Driving_time_s',	
                               'Distance_m',	'Driving_time_s_p',	'Distance_m_p'
                              ])

NameError: ignored

In [ ]:
data_ist_fil.loc[(data_ist_fil['date']=='13.03.2019') & (data_ist_fil['UML']==88094)][100:150]

,,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST,Ank_ZT_PLAN,S_Station,E_Station,NHaltestelle,Node A to Node B,date,Ab_ZT_ACTUAL,Ank_ZT_ACTUAL,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
230868,7887840,13.03.2019,88094.0,1504.0,2019-03-13 10:20:00,28.0,U Straßburger Straße,9.2,17.2,32.2,27.3,0,19.4,2019-03-13 10:21:00,S Rübenkamp,Wandsbeker Allee,Wandsbeker Allee,U Straßburger Straße|Wandsbeker Allee,13.03.2019,2019-03-13 10:29:12,2019-03-13 10:30:06,54.0,291.000000,60.0,286.666667
230869,7887841,13.03.2019,88094.0,1504.0,2019-03-13 10:21:00,28.0,Wandsbeker Allee,9.1,18.1,-,-,0,14.5,2019-03-13 10:47:00,S Rübenkamp,Wandsbeker Allee,Source,Wandsbeker Allee|Source,13.03.2019,2019-03-13 10:30:06,2019-03-13 10:46:36,990.0,3987.500000,1560.0,7843.333333
230870,7887842,13.03.2019,88094.0,1504.0,2019-03-13 10:47:00,28.0,U Wandsbek Markt,-0.4,18.9,6.8,6.4,0,5.9,2019-03-13 10:50:00,U Wandsbek Markt,S Rübenkamp,Wandsbeker Allee,U Wandsbek Markt|Wandsbeker Allee,13.03.2019,2019-03-13 10:46:36,2019-03-13 10:56:24,588.0,963.666667,180.0,945.000000
230871,7887843,13.03.2019,88094.0,1504.0,2019-03-13 10:50:00,28.0,Wandsbeker Allee,6.4,10.7,11.4,9.6,0,9.3,2019-03-13 10:52:00,U Wandsbek Markt,S Rübenkamp,U Straßburger Straße,Wandsbeker Allee|U Straßburger Straße,13.03.2019,2019-03-13 10:56:24,2019-03-13 10:58:36,132.0,341.000000,120.0,356.666667
230872,7887844,13.03.2019,88094.0,1504.0,2019-03-13 10:52:00,28.0,U Straßburger Straße,6.6,15.7,44.9,39.3,0,26.2,2019-03-13 10:54:00,U Wandsbek Markt,S Rübenkamp,U Alter Teichweg,U Straßburger Straße|U Alter Teichweg,13.03.2019,2019-03-13 10:58:36,2019-03-13 10:59:48,72.0,524.000000,120.0,523.333333
230873,7887845,13.03.2019,88094.0,1504.0,2019-03-13 10:54:00,28.0,U Alter Teichweg,5.8,19.1,7.1,5.3,0,6.7,2019-03-13 10:57:00,U Wandsbek Markt,S Rübenkamp,Habichtstraße (Mitte),U Alter Teichweg|Habichtstraße (Mitte),13.03.2019,2019-03-13 10:59:48,2019-03-13 11:08:24,516.0,960.333333,180.0,955.000000
230874,7887846,13.03.2019,88094.0,1504.0,2019-03-13 10:57:00,28.0,Habichtstraße (Mitte),11.4,11.3,23.5,37.7,0,11.9,2019-03-13 10:58:00,U Wandsbek Markt,S Rübenkamp,U Habichtstraße,Habichtstraße (Mitte)|U Habichtstraße,13.03.2019,2019-03-13 11:08:24,2019-03-13 11:09:24,60.0,198.333333,60.0,188.333333
230875,7887847,13.03.2019,88094.0,1504.0,2019-03-13 10:58:00,28.0,U Habichtstraße,11.4,12.4,33.7,29.2,0,25.6,2019-03-13 11:00:00,U Wandsbek Markt,S Rübenkamp,Habichtsplatz,U Habichtstraße|Habichtsplatz,13.03.2019,2019-03-13 11:09:24,2019-03-13 11:10:12,48.0,341.333333,120.0,413.333333
230876,7887848,13.03.2019,88094.0,1504.0,2019-03-13 11:00:00,28.0,Habichtsplatz,10.2,13.9,17.6,15.3,50,15.6,2019-03-13 11:02:00,U Wandsbek Markt,S Rübenkamp,Neue Wöhr,Habichtsplatz|Neue Wöhr,13.03.2019,2019-03-13 11:10:12,2019-03-13 11:12:00,108.0,468.000000,120.0,463.333333
230877,7887849,13.03.2019,88094.0,1504.0,2019-03-13 11:02:00,28.0,Neue Wöhr,10.0,19.0,34.6,29.0,23,21.9,2019-03-13 11:03:00,U Wandsbek Markt,S Rübenkamp,Hartzloh,Neue Wöhr|Hartzloh,13.03.2019,2019-03-13 11:12:00,2019-03-13 11:13:00,60.0,365.000000,60.0,316.666667


In [ ]:
data_ist_fil['Ab_ZT_PLAN'] = data_ist_fil['Ab_ZT_PLAN'].astype('datetime64[ns]')

In [ ]:
data_ist_fil.dtypes

Datum                           object
UML                            float64
Wagen                          float64
Ab_ZT_PLAN              datetime64[ns]
Linie                           object
Haltestelle                     object
Fahrplanlage_IST               float64
v_PLAN                         float64
v_IST_S2S1                      object
v_netto_S2S1                    object
FFVP_GESCHWINDIGKEIT            object
v_IST                           object
Ank_ZT_PLAN                     object
S_Station                       object
E_Station                       object
NHaltestelle                    object
Node A to Node B                object
date                            object
Ab_ZT_ACTUAL                    object
Ank_ZT_ACTUAL                   object
Driving_time_s                 float64
Distance_m                     float64
Driving_time_s_p               float64
Distance_m_p                   float64
dtype: object

In [ ]:
data_ist_fil.loc[data_ist_fil['Ab_ZT_PLAN'].dt.month <7]

,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST,Ank_ZT_PLAN,S_Station,E_Station,NHaltestelle,Node A to Node B,date,Ab_ZT_ACTUAL,Ank_ZT_ACTUAL,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
2,01.05.2019,88001.0,1506.0,2019-05-01 04:08:00,171,Eckerkoppel,0.1,36.1,49.2,45.2,35,44.2,2019-05-01 04:09:00,U Farmsen,Straßburger Platz,Ebeersreye,Eckerkoppel|Ebeersreye,01.05.2019,2019-05-01 04:08:06,2019-05-01 04:08:54,48.0,589.333333,60.0,601.666667
3,01.05.2019,88001.0,1506.0,2019-05-01 04:09:00,171,Ebeersreye,-0.1,20.1,14.3,13.2,38,13.9,2019-05-01 04:11:00,U Farmsen,Straßburger Platz,Rentenversicherung Nord,Ebeersreye|Rentenversicherung Nord,01.05.2019,2019-05-01 04:08:54,2019-05-01 04:11:48,174.0,671.833333,120.0,670.000000
4,01.05.2019,88001.0,1506.0,2019-05-01 04:11:00,171,Rentenversicherung Nord,0.8,19.1,47.9,41.4,45,30.2,2019-05-01 04:12:00,U Farmsen,Straßburger Platz,Sylter Weg,Rentenversicherung Nord|Sylter Weg,01.05.2019,2019-05-01 04:11:48,2019-05-01 04:12:30,42.0,352.333333,60.0,318.333333
5,01.05.2019,88001.0,1506.0,2019-05-01 04:12:00,171,Sylter Weg,0.5,28.4,18.5,16.2,0,16.3,2019-05-01 04:13:00,U Farmsen,Straßburger Platz,Am Stadtrand,Sylter Weg|Am Stadtrand,01.05.2019,2019-05-01 04:12:30,2019-05-01 04:14:06,96.0,434.666667,60.0,473.333333
6,01.05.2019,88001.0,1506.0,2019-05-01 04:13:00,171,Am Stadtrand,1.1,21.2,50.8,44.5,43,43.8,2019-05-01 04:14:00,U Farmsen,Straßburger Platz,Helbingtwiete,Am Stadtrand|Helbingtwiete,01.05.2019,2019-05-01 04:14:06,2019-05-01 04:14:36,30.0,365.000000,60.0,353.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18903030,25.02.2019,88089.0,1632.0,2019-02-25 09:54:00,8,Pfeilshofer Weg,1.1,10.8,18.2,15.4,0,12.8,2019-02-25 09:56:00,S Poppenbüttel (Stormarnplatz),S Poppenbüttel (Stormarnplatz),Rolfinckstraße,Pfeilshofer Weg|Rolfinckstraße,25.02.2019,2019-02-25 09:55:06,2019-02-25 09:56:48,102.0,362.666667,120.0,360.000000
18903031,25.02.2019,88089.0,1632.0,2019-02-25 09:56:00,8,Rolfinckstraße,0.8,27.0,33.4,31.0,0,19.2,2019-02-25 09:58:00,S Poppenbüttel (Stormarnplatz),S Poppenbüttel (Stormarnplatz),Am Damm,Rolfinckstraße|Am Damm,25.02.2019,2019-02-25 09:56:48,2019-02-25 09:59:36,168.0,896.000000,120.0,900.000000
18903032,25.02.2019,88089.0,1632.0,2019-02-25 09:58:00,8,Am Damm,1.6,33.5,47.8,42.8,0,34.0,2019-02-25 09:59:00,S Poppenbüttel (Stormarnplatz),S Poppenbüttel (Stormarnplatz),Maisredder,Am Damm|Maisredder,25.02.2019,2019-02-25 09:59:36,2019-02-25 10:00:24,48.0,453.333333,60.0,558.333333
18903033,25.02.2019,88089.0,1632.0,2019-02-25 09:59:00,8,Maisredder,1.4,26.2,50.6,45.4,43,27.1,2019-02-25 10:00:00,S Poppenbüttel (Stormarnplatz),S Poppenbüttel (Stormarnplatz),Königsberger Straße,Maisredder|Königsberger Straße,25.02.2019,2019-02-25 10:00:24,2019-02-25 10:01:36,72.0,542.000000,60.0,436.666667


In [ ]:
#filter ONLY first 6 month
data_ist_fil_01_06 = data_ist_fil.loc[data_ist_fil['Ab_ZT_PLAN'].dt.month <2].reset_index(drop = True)
data_ist_fil_01_06

,Datum,UML,Wagen,Ab_ZT_PLAN,Linie,Haltestelle,Fahrplanlage_IST,v_PLAN,v_IST_S2S1,v_netto_S2S1,FFVP_GESCHWINDIGKEIT,v_IST,Ank_ZT_PLAN,S_Station,E_Station,NHaltestelle,Node A to Node B,date,Ab_ZT_ACTUAL,Ank_ZT_ACTUAL,Driving_time_s,Distance_m,Driving_time_s_p,Distance_m_p
0,01.01.2019,88001.0,1213.0,2019-01-01 04:46:00,171.0,U Farmsen,0.5,19.0,9.2,7.8,14,8.4,2019-01-01 04:47:00,U Farmsen,U S Barmbek,Eissporthalle,U Farmsen|Eissporthalle,01.01.2019,2019-01-01 04:46:30,2019-01-01 04:48:48,138.0,322.000000,60.0,316.666667
1,01.01.2019,88001.0,1213.0,2019-01-01 04:47:00,171.0,Eissporthalle,1.8,25.3,39.9,35.8,18,17.6,2019-01-01 04:48:00,U Farmsen,U S Barmbek,Eckerkoppel,Eissporthalle|Eckerkoppel,01.01.2019,2019-01-01 04:48:48,2019-01-01 04:50:18,90.0,440.000000,60.0,421.666667
2,01.01.2019,88001.0,1213.0,2019-01-01 04:48:00,171.0,Eckerkoppel,2.3,36.1,38.0,34.6,0,24.0,2019-01-01 04:49:00,U Farmsen,U S Barmbek,Ebeersreye,Eckerkoppel|Ebeersreye,01.01.2019,2019-01-01 04:50:18,2019-01-01 04:51:48,90.0,600.000000,60.0,601.666667
3,01.01.2019,88001.0,1213.0,2019-01-01 04:49:00,171.0,Ebeersreye,2.8,20.1,16.8,15.5,0,14.6,2019-01-01 04:51:00,U Farmsen,U S Barmbek,Rentenversicherung Nord,Ebeersreye|Rentenversicherung Nord,01.01.2019,2019-01-01 04:51:48,2019-01-01 04:54:36,168.0,681.333333,120.0,670.000000
4,01.01.2019,88001.0,1213.0,2019-01-01 04:51:00,171.0,Rentenversicherung Nord,3.6,19.1,45.9,38.3,0,22.1,2019-01-01 04:52:00,U Farmsen,U S Barmbek,Sylter Weg,Rentenversicherung Nord|Sylter Weg,01.01.2019,2019-01-01 04:54:36,2019-01-01 04:55:24,48.0,294.666667,60.0,318.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983737,26.01.2019,88019.0,1633.0,2019-01-26 05:33:00,11,Schimmelmannstraße (Mitte),0.2,14.3,31.7,28.5,44,26.3,2019-01-26 05:35:00,Am Hegen,U Wandsbek Markt,Ziethenstraße,Schimmelmannstraße (Mitte)|Ziethenstraße,26.01.2019,2019-01-26 05:33:12,2019-01-26 05:34:24,72.0,526.000000,120.0,476.666667
983738,26.01.2019,88019.0,1633.0,2019-01-26 05:35:00,11,Ziethenstraße,-0.6,42.2,46.9,42.7,0,34.7,2019-01-26 05:36:00,Am Hegen,U Wandsbek Markt,Birtstraße,Ziethenstraße|Birtstraße,26.01.2019,2019-01-26 05:34:24,2019-01-26 05:35:24,60.0,578.333333,60.0,703.333333
983739,26.01.2019,88019.0,1633.0,2019-01-26 05:36:00,11,Kedenburgstraße,0.2,17.3,17.5,15.6,0,15.3,2019-01-26 05:38:00,Am Hegen,U Wandsbek Markt,Josephstraße,Kedenburgstraße|Josephstraße,26.01.2019,2019-01-26 05:36:12,2019-01-26 05:38:18,126.0,535.500000,120.0,576.666667
983740,26.01.2019,88019.0,1633.0,2019-01-26 05:38:00,11,Josephstraße,0.3,16.0,27.4,24.3,30,24.2,2019-01-26 05:41:00,Am Hegen,U Wandsbek Markt,U Wandsbek Markt,Josephstraße|U Wandsbek Markt,26.01.2019,2019-01-26 05:38:18,2019-01-26 05:40:06,108.0,726.000000,180.0,800.000000


In [ ]:
#data.to_csv('restructured_hochbahn.csv')
data_ist_fil.to_csv('IST_filter_01.csv')
!cp IST_filter_01.csv "/content/drive/My Drive/Colab Notebooks/Thesis"

In [ ]:
!cp IST_filter_01.csv "/content/drive/My Drive/Colab Notebooks/Thesis"